In [28]:
import pandas as pd
import numpy as np 
import pickle
import json

def read_data():
    
    with open('instances.jsonl','r') as f:
        instances = []
        for line in f:
        
            instances.append(json.loads(line))
    with open('truth.jsonl', 'r') as f:
        labels = []
        for line in f:
            labels.append(json.loads(line))
    df = pd.DataFrame(instances)
    df = df.set_index('id')
    labels = pd.DataFrame(labels)
    labels = labels.set_index('id')
    df['label']  = 0
    df['label'][labels['truthClass'] == 'clickbait'] = 1
    df['truthJudgments'] = labels.truthJudgments
    df['postText'] = df['postText'].apply(lambda x: x[0])
    df.index = df.index.astype(int)
    return df

In [29]:
with open('results','rb') as f:
    results = pickle.load(f)

In [30]:
df = read_data()
df.head()

/Users/tianyuyang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,postMedia,postText,postTimestamp,targetCaptions,targetDescription,targetKeywords,targetParagraphs,targetTitle,label,truthJudgments
id,,,,,,,,,,
858462320779026433,[],UK’s response to modern slavery leaving victim...,Sat Apr 29 23:25:41 +0000 2017,[modern-slavery-rex.jpg],“Inexcusable” failures in the UK’s system for ...,"modern slavery, Department For Work And Pensio...",[Thousands of modern slavery victims have not ...,‘Inexcusable’ failures in UK’s response to mod...,0,"[0.3333333333, 0.0, 0.3333333333, 0.0, 0.0]"
858421020331560960,[],this is good,Sat Apr 29 20:41:34 +0000 2017,"[In this July 1, 2010 file photo, Dr. Charmain...",President Donald Trump has appointed pro-life ...,"Americans United for Life, Dr. Charmaine Yoest...",[President Donald Trump has appointed the pro-...,Donald Trump Appoints Pro-Life Advocate as Ass...,1,"[1.0, 1.0, 1.0, 1.0, 1.0]"
858368123753435136,[],"The ""forgotten"" Trump roast: Relive his brutal...",Sat Apr 29 17:11:23 +0000 2017,[President Trump will not attend this year's W...,President Trump won't be at this year's White ...,"trump whcd, whcd, white house correspondents d...",[When the White House correspondents’ dinner i...,The ‘forgotten’ Trump roast: Relive his brutal...,0,"[0.3333333333, 1.0, 0.3333333333, 0.0, 0.66666..."
858323428260139008,[],Meet the happiest #dog in the world!,Sat Apr 29 14:13:46 +0000 2017,"[Maru , Maru, Maru, Maru, Maru]","The article is about Maru, a husky dog who has...","Maru, husky, dogs, pandas, furball, instagram",[Adorable is probably an understatement. This ...,"Meet The Happiest Dog In The World, Maru The H...",1,"[1.0, 0.6666666666, 1.0, 1.0, 1.0]"
858283602626347008,[],Tokyo's subway is shut down amid fears over an...,Sat Apr 29 11:35:31 +0000 2017,[All nine lines of Tokyo's subway system were ...,"The temporary suspension, which lasted ten min...","Tokyo,subway,shut,fears,North,Korean,attack",[One of Tokyo's major subways systems says it ...,Tokyo's subway is shut down amid fears over an...,0,"[0.0, 0.0, 0.0, 0.0, 0.0]"


In [4]:
pd.DataFrame(results['bag_of_words']).to_csv('bag_of_words.csv')
pd.DataFrame(results['extracted_features']).to_csv('extracted.csv')
pd.DataFrame(results['combined']).to_csv('combined.csv')

In [33]:
bow_fn = results['bag_of_words']['XGB_test_scores']['fn']
bow_fp = results['bag_of_words']['XGB_test_scores']['fp']
ef_fn = results['extracted_features']['XGB_test_scores']['fn']
ef_fp = results['extracted_features']['XGB_test_scores']['fp']
c_fn = results['combined']['XGB_test_scores']['fn']
c_fp = results['combined']['XGB_test_scores']['fp']

In [34]:
# get unique error cases for each feature selection
bow_fn_unq =bow_fn.difference(ef_fn.union(c_fn))
bow_fp_unq =bow_fp.difference(ef_fp.union(ef_fp))
ef_fn_unq = ef_fn.difference(bow_fn.union(c_fn))
ef_fp_unq = ef_fn.difference(bow_fp.union(c_fp))
c_fn_unq = c_fn.difference(bow_fn.union(ef_fn))
c_fp_unq = c_fp.difference(bow_fp.union(ef_fp))

In [35]:
bow_fn_unq = df.loc[bow_fn_unq,:]
bow_fp_unq = df.loc[bow_fp_unq,:]
ef_fn_unq = df.loc[ef_fn_unq,:]
ef_fp_unq = df.loc[ef_fp_unq,:]
c_fn_unq = df.loc[c_fn_unq,:]
c_fp_unq = df.loc[c_fp_unq,:]

In [37]:
bow_fn_unq.head()

,postMedia,postText,postTimestamp,targetCaptions,targetDescription,targetKeywords,targetParagraphs,targetTitle,label,truthJudgments
id,,,,,,,,,,
804975993039495168,[],32 years and still no justice,Sat Dec 03 09:10:07 +0000 2016,"[Protest in Bhopal, Protest in Bhopal, Protest...",Even After 32 Years The Bhopal Gas Tragedy Vic...,"Bhopal Gas Tragedy, Bhopal Gas Tragedy Anniver...",[For the 5.5 lakh survivors of the Bhopal gas ...,"Even After 32 Years, The Bhopal Gas Tragedy Vi...",1,"[0.6666666666, 1.0, 0.3333333333, 0.6666666666..."
805359024229249024,[],20 holiday makeup looks via @YahooBeauty,Sun Dec 04 10:32:08 +0000 2016,"[<p></p>, <p>Give your traditional smoky eye a...",Want a gold smoky eye for Christmas dinner wit...,,"[More, From now until the new year, you’ll pro...","20 Holiday Makeup Looks, Because Sometimes a G...",1,"[0.6666666666, 0.0, 0.6666666666, 1.0, 1.0]"
805388435171196928,[media/photo_804822334804791296.jpg],25 amazingly cool gadget gifts you may want to...,Sun Dec 04 12:29:00 +0000 2016,[A smart scale that will help you bake a recip...,,,"[They may have a fancy phone, but do they have...",25 Amazingly Cool Gadget Gifts You May Want To...,1,"[0.6666666666, 1.0, 0.3333333333, 0.3333333333..."
805471710812184576,[],The 10 best songs that sample @TearsForFears,Sun Dec 04 17:59:55 +0000 2016,[Peter Noble/Redferns Curt SmithÂ and Roland O...,The Weeknd is far from the only artist from th...,,[The new-wave fixation of Abel Tesfaye has exi...,The Ten Best Songs That Sample Tears for Fears,1,"[0.6666666666, 1.0, 0.6666666666, 0.6666666666..."
805637361807851520,[media/photo_805637359341629442.jpg],What's the deal with that video of a kangaroo ...,Mon Dec 05 04:58:09 +0000 2016,"[The footage that started it all., 2016%2f09%2...","Yes, Australia is just as wild as you think it...","animals, kangaroo, video, australia, watercooler",[Seen that viral video of man punching a kanga...,Video of guy punching a kangaroo in the face h...,1,"[1.0, 1.0, 1.0, 0.0, 0.0]"


In [38]:
bow_fp_unq.head()

,postMedia,postText,postTimestamp,targetCaptions,targetDescription,targetKeywords,targetParagraphs,targetTitle,label,truthJudgments
id,,,,,,,,,,
804785219710701569,[],Advertisers have invaded schools and national ...,Fri Dec 02 20:32:03 +0000 2016,[Credit Tim Enthoven],Advertising and intrusive marketing is invadin...,"Advertising and Marketing,Online Advertising,P...","[This year, parks in several states including ...",Mother Nature Is Brought to You By ...,0,"[1.0, 0.3333333333, 0.0, 0.0, 0.0]"
804865957072109572,[],"“Now that he’s president-elect, you just hope ...",Sat Dec 03 01:52:52 +0000 2016,"[Shaquille O'Neal, who won four NBA Championsh...","Shaq: Trump ‘Won Fair and Square,’ Time to ‘Gi...","Donald Trump, NBA, shaquille o'neal,Breitbart ...","[“The Big Aristotle” has spoken, and recount-w...","Shaq: Trump ‘Won Fair and Square,’ Time to ‘Gi...",0,"[0.0, 0.3333333333, 0.0, 0.3333333333, 0.66666..."
805079155905875972,[media/photo_805079152487301120.jpg],Hawaii is expecting snow -- more than two feet...,Sat Dec 03 16:00:03 +0000 2016,"[New Fire TV App, Snow at the Mauna Kea summit...",The summits of Hawaii’s Big Island could get m...,"Mauna Kea, snow, Mauna Loa, hawaii snow, weath...",[HONOLULU -- The summits of Hawaii’s Big Islan...,"Yes, it snows in Hawaii: More than 2 feet of s...",0,"[0.3333333333, 1.0, 0.0, 0.0, 0.6666666666]"
807030514049761280,[media/photo_807030510354595840.jpg],New tool lets you customize what kinds of onli...,Fri Dec 09 01:14:03 +0000 2016,[2016%2f09%2f15%2f63%2fhttpsd2mhye01h4nj2n.clo...,The Rubicon Project's new tool lets you custom...,"ad-blockers, online-ads, ads, business, advert...","[If you're an average web user, chances are yo...",You may soon be able to pick and choose the on...,0,"[1.0, 0.3333333333, 0.0, 0.3333333333, 0.0]"
808076405112274944,[],This account points out everything wrong with ...,Sun Dec 11 22:30:02 +0000 2016,[nipples1.jpg],There’s something wrong with women’s nipples. ...,",",[There’s something wrong with women’s nipples....,This account points out everything wrong with ...,0,"[0.3333333333, 1.0, 0.3333333333, 1.0, 0.33333..."


In [39]:
ef_fn_unq.head()

,postMedia,postText,postTimestamp,targetCaptions,targetDescription,targetKeywords,targetParagraphs,targetTitle,label,truthJudgments
id,,,,,,,,,,
804630179268231168,[media/photo_804630176848117761.jpg],Clever pun photos baffling the internet - how ...,Fri Dec 02 10:15:58 +0000 2016,"[HINT: A famous mythical character, HINT: A fa...",These visual clues will have you scratching yo...,"Can,guess,ridiculous,visual,puns","[A cross nailed to a bale of hay, a bunch of m...",The clever pun pictures (that aren't as easy a...,1,"[1.0, 1.0, 0.6666666666, 0.6666666666, 1.0]"
804812045480771584,[],Why would a cereal #brand with such lofty valu...,Fri Dec 02 22:18:38 +0000 2016,"[Kelloggs-George-Soros-AP-Getty, dumpkelloggs,...",The W.K. Kellogg Foundation has partnered with...,"Kellogg, Kellogg Company, Kellogg's, Kellogg's...",[The institutional left’s funding behemoth W.K...,Kellogg Foundation Gave Big to Soros Organizat...,1,"[0.0, 1.0, 1.0, 1.0, 1.0]"
805541350598987777,[],"Barely above the margin of error. Yet ""nearly ...",Sun Dec 04 22:36:38 +0000 2016,"[GettyImages-1161124 (1), Speaker Ryan Retreat...",Only four per cent of British Muslims believe ...,"9/11 Attacks, Al Qaeda, islam in uk,Breitbart ...",[Only four per cent of British Muslims believe...,Only 1-in-25 UK Muslims Believe Al Qaeda Carri...,1,"[1.0, 0.0, 0.3333333333, 0.6666666666, 0.66666..."
806494290969063424,[],Woman sets up camera in bedroom to find out wh...,Wed Dec 07 13:43:17 +0000 2016,"[flatmate3-1.png, flatmate1.png, flatmate2.png...",One thing that most of Generation Y has in com...,",",[One thing that most of Generation Y has in co...,Woman sets up camera to find out what flatmate...,1,"[1.0, 0.3333333333, 0.3333333333, 1.0, 0.66666..."
806501761259343873,[],This #Harvard student’s inspiring #Facebook po...,Wed Dec 07 14:12:58 +0000 2016,"[mgr, jayalalitha]",This article is about Harvard student's inspir...,"charanya kanna, MGR, Jayalalitha","[Charanya Kannan, a Harvard student, posted an...",This Harvard Student’s Inspiring Facebook Post...,1,"[0.6666666666, 1.0, 1.0, 1.0, 1.0]"


In [40]:
ef_fp_unq.head()

,postMedia,postText,postTimestamp,targetCaptions,targetDescription,targetKeywords,targetParagraphs,targetTitle,label,truthJudgments
id,,,,,,,,,,
804143133197881344,[],Should Australia be burning rubbish for electr...,Thu Dec 01 02:00:37 +0000 2016,"[Rubbish fills a rubbish dump, Launch Radio St...",There are some who believe we should burn our ...,"waste-to-energy, power, electricity, world bio...",[There are some who believe Australia should b...,Should Australia be burning rubbish for electr...,1,"[0.6666666666, 0.6666666666, 0.6666666666, 0.6..."
804180781694390272,[],"Visit Myanmar’s stunning capital now, while th...",Thu Dec 01 04:30:13 +0000 2016,[U.S. Stocks Post Biggest One-Day Rally Since ...,Aung San Suu Kyi’s government inherited remote...,"burma,capital,dystopia,naypyidaw,naypyitaw,pol...","[NAYPYITAW, Myanmar—Tourists like Petter Sørli...",Visit Myanmar’s Capital Now! There’s Still a L...,1,"[0.6666666666, 1.0, 0.3333333333, 0.6666666666..."
804194231283814400,[media/photo_804194229140488192.jpg],Here’s the price each major oil producing coun...,Thu Dec 01 05:23:40 +0000 2016,[Oils Markets Await OPEC Agreement Outcome],OPEC surprised the market in September with a ...,world,[OPEC surprised the market in September with a...,Saudis Temper Call for Cuts and Russia Digs In,1,"[0.3333333333, 0.6666666666, 1.0, 0.6666666666..."
804330494120824832,[media/photo_804330492124364800.jpg],[SLIDE DECK] The retail industry is on the cus...,Thu Dec 01 14:25:08 +0000 2016,[],BI Intelligence has created a slide deck explo...,,"[Nancee Halpin, 20.10.2016, 21:31 171, faceboo...",THE FUTURE OF RETAIL 2016 [SLIDE DECK],1,"[0.6666666666, 0.0, 1.0, 0.6666666666, 0.0]"
804477833208823808,[media/photo_804477830922956800.jpg],Here's Trump's plan to keep jobs in America,Fri Dec 02 00:10:36 +0000 2016,"[Trump-Carrier Deal: Will It Set a Precedent?,...",Presidential jawboning is back. American compa...,"personal-finance,technology,world",[Presidential jawboning is back. American comp...,How Trump Plans to Keep Jobs in America: Press...,1,"[0.6666666666, 1.0, 0.6666666666, 1.0, 0.66666..."


In [41]:
c_fn_unq.head()

,postMedia,postText,postTimestamp,targetCaptions,targetDescription,targetKeywords,targetParagraphs,targetTitle,label,truthJudgments
id,,,,,,,,,,
813834622891081728,[media/photo_813834620496211969.jpg],The best of 2016 and sometimes well before it....,Tue Dec 27 19:51:09 +0000 2016,[],"A selection of books, music, radio and TV with...",,"[A great day for a storming., I’m always so im...",The Best of 2016 and Sometimes Well Before It,1,"[0.6666666666, 1.0, 0.3333333333, 0.6666666666..."
839259842992820224,[],#Vault7 #DeepState #BestAndBrightest,Tue Mar 07 23:41:53 +0000 2017,"[AP Photo/Andrew Harnik, Pence Pushes Paul Rya...",In a press release for their most recent data ...,"CIA, hacking, Julian Assange, WikiLeaks,Nation...",[In a press release for their most recent data...,WikiLeaks Claims CIA Lost Control of ‘Hacking ...,1,"[1.0, 0.3333333333, 1.0, 0.6666666666, 0.33333..."
851096379447549952,[media/photo_851096377098743808.jpg],Fighting Trump? Your best weapon is the one yo...,Sun Apr 09 15:36:04 +0000 2017,[2016%2f06%2f29%2fe7%2fhttpsd2mhye01h4nj2n.clo...,Try this strategy now.,"research, politics, altruism, kindness, scienc...",[The year 2017 has a surprise in store if you ...,Fighting Trump? Your best weapon is the one yo...,1,"[0.6666666666, 1.0, 0.3333333333, 1.0, 0.66666..."
854204597044367360,[],Are Your Instagram Habits Like Everyone Else's?,Tue Apr 18 05:27:00 +0000 2017,[Angelo Spagnolo],,,"[Log in or sign up to create your own posts., ...",Are Your Instagram Habits Like Everyone Else's?,1,"[0.6666666666, 1.0, 1.0, 0.6666666666, 1.0]"
855791049565888512,[media/photo_855476216530219009.jpg],I tried five different period subscription box...,Sat Apr 22 14:31:00 +0000 2017,"[Lara Parker, Instagram, Le Parcel is the chea...",,,"[Log in or sign up to create your own posts., ...",I Tried 5 Different Subscription Boxes For My ...,1,"[1.0, 0.0, 0.6666666666, 1.0, 0.3333333333]"


In [44]:
c_fp_unq.head()

,postMedia,postText,postTimestamp,targetCaptions,targetDescription,targetKeywords,targetParagraphs,targetTitle,label,truthJudgments
id,,,,,,,,,,
809862290145034240,[],POTUS in response to @MarthaRaddatz' question ...,Fri Dec 16 20:46:31 +0000 2016,"[ABC News, ABC News, Lakiesha Howard, Therese ...",,,[POTUS in response to @MarthaRaddatz' question...,"ABC News auf Twitter: ""POTUS in response to @M...",0,"[0.3333333333, 0.0, 0.3333333333, 0.0, 0.0]"
814486480051650561,[media/photo_814486476901728256.jpg],The newest startup trend is fragrances,Thu Dec 29 15:01:23 +0000 2016,"[Subscribe to Bloomberg Businessweek, Photo Il...",Hawthorne and a new batch of scent merchants w...,,"[Fragrances are a $4 billion annual market, wi...",Online Fragrance Startups Smell Money,0,"[0.6666666666, 0.3333333333, 0.3333333333, 0.3..."
821129915386957824,[],Edith Bowman: 'Japan was just this wonderful s...,Mon Jan 16 23:00:02 +0000 2017,"[Radio DJ Edith Bowman , The Big Sur coast alo...",,"celebrity_travel,celebrity_interviews,Celebrit...","[As a family, we really make an effort to trav...",Celebrity interviews Edith Bowman: 'Japan was ...,0,"[0.3333333333, 0.0, 0.0, 0.6666666666, 0.33333..."
823369814924259329,[media/photo_823369812445372416.jpg],Patriots. Falcons.\n\nWho ya got? Vote:,Mon Jan 23 03:20:36 +0000 2017,[],"With the Super Bowl LI matchup set, it's time ...","Super Bowl LI, Atlanta Falcons, New England Pa...",[The conference championships are over and the...,Vote: Who will win Super Bowl LI?,0,"[0.3333333333, 0.6666666666, 0.0, 0.0, 0.0]"
823381006145167360,[media/photo_823381003733532673.jpg],Trump on #WomensMarch protesters: ‘Why didn’t ...,Mon Jan 23 04:05:04 +0000 2017,[Protesters gather during the Women's March on...,President Donald Trump questioned the more tha...,,"[More, President Donald Trump questioned the m...",Trump on Women’s March protesters: ‘Why didn’t...,0,"[0.6666666666, 0.0, 0.3333333333, 1.0, 0.33333..."


In [49]:
df.loc[bow_fn.intersection(ef_fn),:].head()

,postMedia,postText,postTimestamp,targetCaptions,targetDescription,targetKeywords,targetParagraphs,targetTitle,label,truthJudgments
id,,,,,,,,,,
856192826241110017,[media/photo_856192815210127362.jpg],Frank Ocean just released another new song and...,Sun Apr 23 17:07:31 +0000 2017,"[Lara Parker, Lens with frank ocean, how i sle...",,,"[Log in or sign up to create your own posts., ...",Frank Ocean Just Released Another New Song And...,1,"[0.6666666666, 0.6666666666, 0.6666666666, 0.0..."
830778487640240128,[],How to dress for Valentine's Day? Hold the rom...,Sun Feb 12 14:00:01 +0000 2017,"[Philosophy SS17, collage 1, How to style a sh...","In the sales, I bought a dress that is inesca...","Lifestyle,Fashion,Other,Standard,Wardrobe solu...","[In the sales, I bought a dress that is inesc...",How to dress for Valentine's Day? Hold the rom...,1,"[1.0, 0.3333333333, 1.0, 0.3333333333, 0.66666..."
821026728667533313,[],Britain's oldest Olympian: Hitler stopped me g...,Mon Jan 16 16:10:00 +0000 2017,[Britain's oldest Olympian: Hitler stopped me ...,Britain's oldest Olympian Bill Lucas celebrate...,"News,World War Two,Standard,Olympics",[Britain's oldest Olympian Bill Lucas celebrat...,Britain's oldest Olympian: Hitler stopped me g...,1,"[0.3333333333, 0.6666666666, 0.6666666666, 0.6..."
822443790938107907,[],ICYMI: Even though he averages a triple-double...,Fri Jan 20 14:00:54 +0000 2017,"[Astronomical seasons, like Westbrook's, shoul...",Thunder star Russell Westbrook was left out in...,"All-Star, starters, ASG, NBA, Russell Westbroo...",[Russell Westbrook is not among the Western Co...,"LeBron James, Stephen Curry lead ASG starters;...",1,"[0.6666666666, 0.3333333333, 0.6666666666, 0.0..."
817324408796835842,[media/photo_817324402132008962.jpg],Katie Piper was burned with acid by her ex-boy...,Fri Jan 06 10:58:18 +0000 2017,"[katiepiper.jpg, confidence-katie-piper.jpg]",Eight years after being the victim of a heinou...,"Katie Piper, Health & Families, Lifestyle",[Model and TV presenter tells the Independent ...,Katie Piper: Model left burned after acid atta...,1,"[0.3333333333, 0.6666666666, 1.0, 0.3333333333..."
